In [1]:
%load_ext autoreload
%autoreload 2

# Proyecto final Bioinformática (NLP)

Mirar el siguiente video. Sobre todo la primer parte como para tener contexto sobre descubrimiento de drogas:

https://www.youtube.com/watch?v=jBlTQjcKuaY

Resumen y puntos importantes del video:
- Queremos entender la bioactividad de una molecúla (molecule_chembl_id) sobre una encima (Acetylcholinesterase)
- La bioactividad se medirá en este caso con el IC50 (standard_value)
- A menor IC50, menos droga para generar la misma actividad, es decir, mayor actividad relativa
- La notación de la fórmula química se llama smiles (https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system)
- Existen distintas técnicas para obtener features de las moléculas y en el video se describen 2:
    - Descriptores de Lipinski
    - Fingerprints del tipo pubchem
- Se construye un modelo de regresión con RandomForest para estimar el pI50 (IC50 en escala logarítimica) dado los fingerprints de entrada

### Objetivos del proyecto:
- Evaluar distintas alternativas de modelos de deep learning para resolver este problema
    - LSTM
    - CNN
    - TextCNN
- Mejorar la métrica del RandomForest
- En vez de ingresar con los features de entrada (fingerprints) como en el video, utilizar técnicas de embeddings usuales en NLP
    - Tokenización en modo caracter dado el smiles de la fórmula química
    - Utilizando un tokenizer sobre los smiles
    - Puedo usar técnicas modernas de tokenización (https://deepchem.readthedocs.io/en/2.4.0/api_reference/tokenizers.html)
- La salida a estimar por el modelo será el pIC50
- La métrica, para comparar con los resultados del RandomForest será el $R^2$

### Librerías:
- chembl-webresource-client: Para bajar el dataset (https://pypi.org/project/chembl-webresource-client/)
- deepchem: libería muy interesante con muchas implementaciones de deep learning aplicadas a la química (https://github.com/deepchem/deepchem)

In [3]:
! pip install chembl_webresource_client

  Created wheel for easydict: filename=easydict-1.10-py3-none-any.whl size=6497 sha256=9712ea2c51ba36cad4d7c8692f06b8e2cee4b655c948e03eb22676d5441f08e1
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\fe\4e\02\c9c3154e4845bfdbf1fdf344f5a89f16dcbb4f627a908c9974
Successfully built easydict
  Attempting uninstall: itsdangerous
    Found existing installation: itsdangerous 1.1.0
    Uninstalling itsdangerous-1.1.0:
      Successfully uninstalled itsdangerous-1.1.0
  Attempting uninstall: attrs
    Found existing installation: attrs 20.3.0
    Uninstalling attrs-20.3.0:
      Successfully uninstalled attrs-20.3.0


In [4]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

# Cliente API

En esta notebook solo se baja el dataset. No tiene que hacer nada más que ejecutarla y entenderla

Librería para baja el dataset

In [5]:
target = new_client.target
target_query = target.search('acetylcholinesterase')
targets = pd.DataFrame.from_dict(target_query)

In [6]:
targets.head()

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,"[{'xref_id': 'P22303', 'xref_name': None, 'xre...",Homo sapiens,Acetylcholinesterase,27.0,False,CHEMBL220,"[{'accession': 'P22303', 'component_descriptio...",SINGLE PROTEIN,9606
1,[],Homo sapiens,Cholinesterases; ACHE & BCHE,27.0,False,CHEMBL2095233,"[{'accession': 'P06276', 'component_descriptio...",SELECTIVITY GROUP,9606
2,[],Drosophila melanogaster,Acetylcholinesterase,18.0,False,CHEMBL2242744,"[{'accession': 'P07140', 'component_descriptio...",SINGLE PROTEIN,7227
3,[],Bemisia tabaci,AChE2,16.0,False,CHEMBL2366409,"[{'accession': 'B3SST5', 'component_descriptio...",SINGLE PROTEIN,7038
4,[],Leptinotarsa decemlineata,Acetylcholinesterase,16.0,False,CHEMBL2366490,"[{'accession': 'Q27677', 'component_descriptio...",SINGLE PROTEIN,7539


In [7]:
selected_target = targets.target_chembl_id[0]
selected_target
# 'CHEMBL220'

'CHEMBL220'

In [8]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

# Bajada de data
Puede tardar un poco dependiendo de que tan saturado este el server

Por eso el for, para ver el progreso y bajar la ansiedad. Son en el orden de 7500K

In [9]:
res_cols = []
for i, r in enumerate(res):
    print(f'{i}\r', end='')
    res_cols.append(r)

In [17]:
df = pd.DataFrame(res_cols)

In [18]:
df.head(2)

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,33969,[],CHEMBL643384,Inhibitory concentration against acetylcholine...,B,None,None,BAO_0000190,BAO_0000357,...,Homo sapiens,Acetylcholinesterase,9606,None,None,IC50,uM,UO_0000065,None,0.75
1,None,37563,[],CHEMBL643384,Inhibitory concentration against acetylcholine...,B,None,None,BAO_0000190,BAO_0000357,...,Homo sapiens,Acetylcholinesterase,9606,None,None,IC50,uM,UO_0000065,None,0.1


# Limpio data

In [19]:
df = df.dropna(subset=['standard_value', 'canonical_smiles'])
df = df.drop_duplicates(['canonical_smiles'])

In [15]:
df['standard_value'].describe()

count    5.824000e+03
mean     2.588482e+12
std      1.068126e+14
min      0.000000e+00
25%      1.470000e+02
50%      2.400000e+03
75%      1.700000e+04
max      5.888437e+15
Name: standard_value, dtype: float64

In [20]:
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df = df[selection]

In [21]:
df

,molecule_chembl_id,canonical_smiles,standard_value
0,CHEMBL133897,CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1,750.0
1,CHEMBL336398,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC1CC1,100.0
2,CHEMBL131588,CN(C(=O)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F)c1ccccc1,50000.0
3,CHEMBL130628,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F,300.0
4,CHEMBL130478,CSc1nc(-c2ccc(OC(F)(F)F)cc2)nn1C(=O)N(C)C,800.0
...,...,...,...
8384,CHEMBL4859103,CC1=CC2Cc3nc4cc(Cl)ccc4c(NCCCCCCCCCNC(=O)c4cc(...,2.57
8385,CHEMBL4863615,CC1=CC2Cc3nc4cc(Cl)ccc4c(NCCCCCCCCC(=O)Nc4cn[n...,2.39
8386,CHEMBL4854913,CC1=CC2Cc3nc4cc(Cl)ccc4c(NCCCCCCCCC(=O)N4CCNC(...,0.41
8387,CHEMBL4848527,CC1=CC2Cc3nc4cc(Cl)ccc4c(NCCCCCCCCC(=O)NCc4cc[...,0.63


# Preprocesamiento y normalización

In [3]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv('data/acetylcholinesterase_02_bioactivity_data_preprocessed.csv')

In [18]:
df['standard_value'] = df['standard_value'].apply(pd.to_numeric)

In [26]:
df['standard_value'].describe()

count    5.824000e+03
mean     2.588482e+12
std      1.068126e+14
min      0.000000e+00
25%      1.470000e+02
50%      2.400000e+03
75%      1.700000e+04
max      5.888437e+15
Name: standard_value, dtype: float64

In [32]:
df['standard_value'].describe()

count    5.824000e+03
mean     2.588482e+12
std      1.068126e+14
min      0.000000e+00
25%      1.470000e+02
50%      2.400000e+03
75%      1.700000e+04
max      5.888437e+15
Name: standard_value, dtype: float64

In [39]:
# En el video se hace esta normalización. No la veo del todo necesaria
df['standard_value_norm'] = df['standard_value'].apply(lambda x: (x==0)*0.0000001 + (x>=0)*x)
#df['standard_value_norm'] = df['standard_value'].apply(lambda x: (x>1e8)*1e8 + (x<=1e8)*x)

In [40]:
df['pIC50'] = df['standard_value_norm'].apply(lambda x: -np.log10(x*(10**-9)))

In [41]:
df['pIC50'].describe()

count    5824.000000
mean        5.810230
std         1.644699
min        -6.770000
25%         4.769551
50%         5.619789
75%         6.832683
max        16.000000
Name: pIC50, dtype: float64

In [42]:
df.head(2)

,molecule_chembl_id,canonical_smiles,standard_value,standard_value_norm,pIC50
0,CHEMBL133897,CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1,750.0,750.0,6.124939
1,CHEMBL336398,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC1CC1,100.0,100.0,7.000000


In [43]:
! mkdir data

Ya existe el subdirectorio o el archivo data.


In [44]:
df.to_csv('data/acetylcholinesterase_02_bioactivity_data_preprocessed.csv', index=False)